# GPT2を用いた質疑応答システム

## はじめに

このノートブックでは、GPT2と呼ばれる大規模言語モデル(LLM)を用いた質疑応答システムを体験できます。<br>
GPT2とは、もともとOpenAI社が開発したLLMであり、今はMITライセンスが付与され全世界に無料公開されています。<br>
昨今有名なChatGPTは、このGPT2をさらに改良したGPT3.5およびGPT4(Plus版)が使われています。

## 使用するモデル

ここでは、OpenAI社または[rinna株式会社](https://corp.rinna.co.jp/)によって学習された、英語または日本語のGPT2モデルを使用します。<br>
これらのモデルは、[MIT license](https://opensource.org/license/mit/)で提供されています。<br>
下記が、今回使用するモデルのドキュメントとなっております。

https://huggingface.co/openai-community/gpt2<br>
https://huggingface.co/rinna/japanese-gpt2-medium

## 1. ライブラリをインストールする

ここでは、必要なライブラリをインストールします。<br>
もしうまく動かない場合は、カーネルをリスタートするとよいみたいです。

参考: https://www.cloudnotes.tech/entry/rinna_gpt_jp

In [ ]:
! pip install transformers
! pip install sentencepiece
! pip install accelerate

## 2. ライブラリをインポートする

ここでは、必要なライブラリをインポートします。<br>
GPUを利用する設定にしていることを確認してから実行してください。<br>
うまく設定できている場合は、以下のセルを実行すると `cuda` と出力されます。

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

## 3. 言語設定

このプログラムを動かすうえで、使用する言語はここで変更できます。<br>
`use_japanese` の値を変更してください。

In [ ]:
# 日本語のGPTモデルを使用する際はTrue, 英語のGPTモデルを使用する際はFalseにしてください。
# If you use Japanese GPT model, set True, otherwise if you use English GPT model, set False.
use_japanese = True

## 4. モデルとトークナイザを定義する

モデルとトークナイザを定義します。<br>
今回は時間（と計算コスト）の都合上、すでに訓練されたモデルを使用します。<br>
ファインチューニングという手法を利用することで、すでに訓練されたモデルの精度をさらに向上させることができます。<br>
ただし、モデルのロードに少々時間がかかります。お待ちください。

In [ ]:
if use_japanese:
    tokenizer = AutoTokenizer.from_pretrained("rinna/japanese-gpt2-medium", use_fast=False)
    tokenizer.do_lower_case = True  # due to some bug of tokenizer config loading

    model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt2-medium").to(device)
else:
    tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
    model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2").to(device)

## 5. 質問を定義する

次の `question` 変数にお好きな質問を設定して、実行してください。

In [ ]:
question = "東京工業大学のいいところは何ですか。"

## 6. 回答を得る

上の質問を定義して、以下のセルを実行すると、回答が得られます。<br>
回答の生成に少々時間がかかりますのでお待ちください。

この5番と6番は何回でも繰り返すことができます。<br>
また、6番のみを繰り返すと、同じ質問でも違う回答を得ることができます。

もし言語設定を変更したい場合は、3番を変更して、その後3番から順番に実行してください。

In [ ]:
text = "{} ".format(question)
token_ids = tokenizer.encode(text, add_special_tokens=False, return_tensors='pt')

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(device),
        do_sample=True,
        max_new_tokens=128,
        temperature=0.7,
        repetition_penalty=1.1,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )

output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1):])
print("-"*10)
print(output)